In [1]:
from PandaStation import *
from PandaGrasping import *

# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])

# Let's do all of our imports here, too.
import numpy as np
import ipywidgets
import pydot
import pydrake.all
import os
from IPython.display import display, SVG
import subprocess


import pydrake.all
from pydrake.geometry import Cylinder, Box
from pydrake.all import (
    RigidTransform, RotationMatrix, AngleAxis, RollPitchYaw, InverseKinematics, MultibodyPlant, Parser,
    FindResourceOrThrow, Solve, PiecewisePolynomial, TrajectorySource, SceneGraph, DiagramBuilder,
    AddMultibodyPlantSceneGraph, LinearBushingRollPitchYaw, MathematicalProgram, AutoDiffXd, GenerateHtml, Role,
    LeafSystem, AbstractValue, PublishEvent, TriggerType, BasicVector, PiecewiseQuaternionSlerp
    )
import pydrake.perception as mut
import open3d as o3d
from ompl import base as ob
from ompl import geometric as og
import time
from enum import Enum

ycb = {"cracker": "drake/manipulation/models/ycb/sdf/003_cracker_box.sdf", 
    "sugar": "drake/manipulation/models/ycb/sdf/004_sugar_box.sdf", 
    "soup": "drake/manipulation/models/ycb/sdf/005_tomato_soup_can.sdf", 
    "mustard": "drake/manipulation/models/ycb/sdf/006_mustard_bottle.sdf", 
    "gelatin": "drake/manipulation/models/ycb/sdf/009_gelatin_box.sdf", 
    "meat": "drake/manipulation/models/ycb/sdf/010_potted_meat_can.sdf",
    "brick": "drake/examples/manipulation_station/models/061_foam_brick.sdf"}

def ycb_resource(name):
    global ycb
    return FindResourceOrThrow(ycb[name])

def random_ycb_resource():
    global ycb
    ycb_items = list(ycb.items())
    index = np.random.randint(0, len(ycb_items))
    return ycb_resource(ycb_items[index][0])

In [8]:
class Stage(Enum):
    INIT = 0
    TO_PREPICK = 1
    TO_PICK = 2
    PICKING = 3
    TO_POSTPICK = 4
    TO_INIT = 5

class ProcessPointcloud(LeafSystem):
    
    def __init__(self, station, v):
        LeafSystem.__init__(self)
        
        camera_left_pcd = self.DeclareAbstractInputPort("camera_left_pcd", AbstractValue.Make(mut.PointCloud()))
        camera_middle_pcd = self.DeclareAbstractInputPort("camera_middle_pcd", AbstractValue.Make(mut.PointCloud()))
        camera_right_pcd = self.DeclareAbstractInputPort("camera_right_pcd", AbstractValue.Make(mut.PointCloud()))
        self.cameras = {'camera_left': camera_left_pcd,
                       'camera_middle': camera_middle_pcd,
                       'camera_right': camera_right_pcd}
        
        self.station = station
        self.plant = self.station.get_multibody_plant()
        self.plant_context = plant.GetMyContextFromRoot(station_context)
        
        self.v = v # meshcat visualizer
        
        self.DeclareAbstractOutputPort("merged_pcd", 
                                       lambda: AbstractValue.Make(o3d.geometry.PointCloud()), 
                                       self.DoCalcOutput)
        
        self.DeclarePeriodicEvent(0.5, 0.25, event = PublishEvent(
                                            trigger_type = TriggerType.kPeriodic,
                                            callback = self.MyPublish))
    
    def process_table_point_cloud(self, context):
        
        margin = 0.001
        # defines space above table
        crop_min = np.array([0.25, -0.5, 0.05+margin])
        crop_max = np.array([1.25, 0.5, 0.5])
        
        merged_pcd = o3d.geometry.PointCloud()
        
        for name,port in list(self.cameras.items()):
            pcd = self.EvalAbstractInput(context, port.get_index()).get_value()
            pcd = create_open3d_point_cloud(pcd)
            
            pcd = pcd.crop(o3d.geometry.AxisAlignedBoundingBox(min_bound = crop_min, max_bound = crop_max))
            pcd.estimate_normals(search_param = o3d.geometry.KDTreeSearchParamHybrid(radius = 0.1, max_nn = 30))
            camera = self.plant.GetModelInstanceByName(name)
            body = self.plant.GetBodyByName("base", camera)
            X_C = self.plant.EvalBodyPoseInWorld(self.plant_context, body)
            pcd.orient_normals_towards_camera_location(X_C.translation())
            
            merged_pcd +=pcd
            
        return merged_pcd.voxel_down_sample(voxel_size = 0.005)
    
    def MyPublish(self, context, event):
        time = context.get_time()
        if not np.isclose(time, 0.25):
            return
        pcd = self.process_table_point_cloud(context)
        #print("time:", time)
        draw_open3d_point_cloud(self.v.vis["cloud"], pcd, size=0.003, normals_scale = 0.01)
    
    def DoCalcOutput(self, context, output):
        pcd = self.process_table_point_cloud(context)
        output.set_value(pcd)
        
        
class FindGrasp(LeafSystem):
    
    def __init__(self, internal_station):
        LeafSystem.__init__(self)
        
        self.nq = 7
        
        self.merged_pcd_input_port = self.DeclareAbstractInputPort("merged_pcd", AbstractValue.Make(o3d.geometry.PointCloud()))   
        self.panda_grasp_position_output_port = self.DeclareVectorOutputPort("panda_grasp_position", BasicVector(self.nq),
                                                                self.DoCalcOutput)
        
        
        self.station = internal_station
        self.hand_env = self.station.get_hand_env()
        self.hand_env_context = self.hand_env.CreateDefaultContext()
        
        self.plant = self.station.get_multibody_plant()
        self.station_context = self.station.CreateDefaultContext()
        self.plant_context = self.station.GetSubsystemContext(self.plant, self.station_context)
        
        self.rng = np.random.default_rng()
        self.q_nominal = np.array([ 0., 0.55, 0., -1.45, 0., 1.58, 0.])
        self.max_samples = 100
        
    def X_WG_to_q(self, X_G_lower, X_G, X_G_upper, no_cols = False, warm_start = np.array([ 0., 0.55, 0., -1.45, 0., 1.58, 0.])):
        
        plant = self.plant
        plant_context = self.plant_context
        
        ik = InverseKinematics(plant, plant_context)
        theta_tol = 0.01
        q_nominal = np.array([ 0., 0.55, 0., -1.45, 0., 1.58, 0.])

        trans_lower = []
        trans_upper = []

        for i in range(3):
            trans_lower.append(min(X_G_lower.translation()[i], X_G_upper.translation()[i]))
            trans_upper.append(max(X_G_lower.translation()[i], X_G_upper.translation()[i]))

        ik.AddPositionConstraint(
                plant.GetFrameByName("panda_hand"),
                np.zeros(3),
                plant.world_frame(),
                trans_lower,
                trans_upper)
        ik.AddOrientationConstraint(
                plant.GetFrameByName("panda_hand"),
                RotationMatrix(),
                plant.world_frame(),
                X_G.rotation(),
                theta_tol)
        if no_cols:
            ik.AddMinimumDistanceConstraint(0, 0.1)

        q = ik.q()
        prog = ik.prog()
        prog.AddQuadraticErrorCost(np.identity(len(q)), q_nominal, q)
        prog.SetInitialGuess(q, warm_start)

        result = Solve(prog)
        #assert result.is_success()
        print("Success?", result.is_success())
        if not result.is_success():
            names = result.GetInfeasibleConstraintNames(prog)
            print("Infeasible Constraints:")
            for n in names:
                print(n)
        return result.GetSolution(q)        

    
    def DoCalcOutput(self, context, output):
        
        cloud = self.EvalAbstractInput(context, self.merged_pcd_input_port.get_index()).get_value()
        costs = []
        X_Gs = []
        
        start_time = time.time()
        
        for i in range(self.max_samples):
            cost, X_G_lower, X_G, X_G_upper = generate_grasp_candidate_antipodal(self.hand_env, 
                                                                                 self.hand_env_context, 
                                                                                 cloud, 
                                                                                 self.rng)
            if np.isfinite(cost):
                costs.append(cost)
                X_Gs.append((X_G_lower, X_G, X_G_upper))
        
        index = np.asarray(costs).argsort()[:1][0]
        cost = costs[index]
        assert np.isfinite(cost), "could not find a valid grasp pose"
        X_G_lower = X_Gs[index][0]
        X_G = X_Gs[index][1]
        X_G_upper = X_Gs[index][2]
        
        q = self.X_WG_to_q(X_G_lower, X_G, X_G_upper, no_cols = True)
        
        output.set_value(q)
        

class Picker(LeafSystem):
    
    def __init__(self, internal_station):
        LeafSystem.__init__(self)
        
        self.nq = 7
        
        self.grasp_position_input_port = self.DeclareVectorInputPort("grasp_position", BasicVector(self.nq))
        self.merged_pcd_input_port = self.DeclareAbstractInputPort("merged_pcd", 
                                                                   AbstractValue.Make(o3d.geometry.PointCloud()))
        self.panda_position_input_port = self.DeclareVectorInputPort("panda_position", 
                BasicVector(self.nq))
        self.panda_position_command_output_port = self.DeclareVectorOutputPort(
                "panda_position_command", BasicVector(self.nq),
                self.DoCalcOutput)
        self.hand_command_output_port = self.DeclareVectorOutputPort("hand_position_command", BasicVector(1),
                                                                    self.CalcHandOutput)
        
        self.plant = internal_station.get_multibody_plant()
        self.panda = self.plant.GetModelInstanceByName("panda")
        self.station_context = internal_station.CreateDefaultContext()
        self.plant_context = self.plant.GetMyContextFromRoot(self.station_context)
        self.scene_graph = internal_station.get_scene_graph()
        self.scene_graph_context = internal_station.GetSubsystemContext(self.scene_graph, self.station_context)
        self.query_output_port = self.scene_graph.GetOutputPort("query")
        
        
        self.q_S = None
        self.q_G = None
        self.q_P = None
        
        self.panda_traj = None
        self.hand_traj = None
        
        self.X_Pregrasp = None
        self.X_Grasp = None
        
        self.stage = Stage.INIT # Stage(0)
        self.setup_time = 0.5
        
        self.q_nominal = np.array([ 0., 0.55, 0., -1.45, 0., 1.58, 0.])
        
        self.p_tol = 0.01*np.ones(3)
        self.theta_tol = 0.05
        
        self.cloud = None
        
        
    def CalcHandOutput(self, context, output):
        time = context.get_time()
        if (self.hand_traj is None) or (time > self.hand_traj.end_time()):
            output.set_value([0])
        else:
            output.set_value(self.hand_traj.value(time).flatten())
        
    def DoCalcOutput(self, context, output):
        time = context.get_time()
        
        if time < self.setup_time: # allow time for objects to settle
            self.q_S = self.EvalVectorInput(context, self.panda_position_input_port.get_index()).get_value()
            output.set_value(self.q_S)
            return
        
        if self.stage == Stage.INIT:
            print("INITIALIZATION")
            #self.cloud = self.EvalAbstractInput(context, self.merged_pcd_input_port.get_index()).get_value()
            self.q_G = self.EvalVectorInput(context, self.grasp_position_input_port.get_index()).get_value()
            self.q_P, self.X_Pregrasp, self.X_Grasp = self.pregrasp_q()
            self.panda_traj = self.rrt_trajectory(self.q_S, time, self.q_P, time + 3)
            self.hand_traj = self.first_order_traj([0.0], time, [0.08], time + 3)
            self.stage = Stage.TO_PREPICK
            print("TO PREPICK")
            
        if self.stage == Stage.TO_PREPICK:
            
            if time <= self.panda_traj.end_time():
                output.set_value(self.panda_traj.value(time).flatten())
            else:
                self.stage = Stage.TO_PICK
                print("TO PICK")
                self.panda_traj = self.rrt_trajectory(self.q_P, time, self.q_G, time + 2)
                #self.panda_traj = self.straight_line_traj(self.q_P, self.X_Pregrasp, time, 
                                                          #self.X_Grasp, time + 2)
                self.hand_traj = self.first_order_traj([0.08], time, [0.08], time +2)
        
        if self.stage == Stage.TO_PICK:
            if time <= self.panda_traj.end_time():
                output.set_value(self.panda_traj.value(time).flatten())
            else:
                self.stage = Stage.PICKING
                print("PICKING")
                self.hand_traj = self.first_order_traj([0.08], time, [0.0], time +2)
                self.panda_traj = self.first_order_traj(self.q_G, time, self.q_G, time +2)
                
        if self.stage == Stage.PICKING:
            if time <= self.panda_traj.end_time():
                output.set_value(self.panda_traj.value(time).flatten())
            else:
                self.stage = Stage.TO_POSTPICK
                print("TO POSTPICK")
                self.panda_traj = self.rrt_trajectory(self.q_G, time, self.q_S, time + 3)
                self.hand_traj = self.first_order_traj([0.0], time, [0.0], time + 3)
                
        if self.stage == Stage.TO_POSTPICK:
            if time <= self.panda_traj.end_time():
                output.set_value(self.panda_traj.value(time).flatten())
            else:
                pass
                
            
    def straight_line_traj(self, q_start, X_start, start_time, X_end, end_time, N = 10):
        # interpolate between X_start and X_end (we want to maintain same gripper oritentation)
        times = np.array([start_time, end_time])
        ps = np.array([X_start.translation(), X_end.translation()])
        p_traj = PiecewisePolynomial.FirstOrderHold(times, ps.T)
        # solve ik problem at each time 
        times = np.linspace(start_time, end_time, N)
        qs = []
        R_WG = X_end.rotation()
        for i,t in enumerate(times):
            p_WG = p_traj.value(t)
            X_WG = RigidTransform(R_WG, p_WG)
            if i == 0:
                warm_start = q_start
            else:
                warm_start = qs[-1]
                
            q = self.X_WG_to_q(X_WG, warm_start = warm_start)
            qs.append(q)
            
        qs = np.array(qs)
        return PiecewisePolynomial.CubicShapePreserving(times, qs.T)
                
    def first_order_traj(self, q_start, start_time, q_end, end_time):
        time = np.array([start_time, end_time])
        qs = np.array([q_start, q_end])
        return PiecewisePolynomial.FirstOrderHold(time, qs.T)
            
    def rrt_trajectory(self, q_start, start_time, q_goal, goal_time):

        joint_limits = self.joint_limits() 

        space = ob.RealVectorStateSpace(self.nq)
        bounds = ob.RealVectorBounds(self.nq)

        for i in range(self.nq):
            bounds.setLow(i, joint_limits[i][0])
            bounds.setHigh(i, joint_limits[i][1])

        space.setBounds(bounds)
        ss = og.SimpleSetup(space)
        ss.setStateValidityChecker(ob.StateValidityCheckerFn(self.isStateValid))
        start = self.q_to_state(space, q_start)
        assert self.isStateValid(start), "you have supplied an invalid start state RRT"
        goal = self.q_to_state(space, q_goal)
        assert self.isStateValid(goal), "you have supplied an invalid goal state for RRT"
        ss.setStartAndGoalStates(start, goal) 
        solved = ss.solve()

        assert solved

        ss.simplifySolution()
        path = ss.getSolutionPath()

        res = []
        costs = []
        for state in path.getStates():
            q = self.parse_state(state)
            if len(costs) == 0:
                costs.append(0)
            else:
                d = q - res[-1]
                cost = np.sqrt(d.dot(d))
                costs.append(costs[-1] + cost)
            res.append(q)
        
        
        if (costs[-1] == 0):
            costs[-1] = 1

        qs = np.array(res)
        times = start_time + ((goal_time - start_time)*np.array(costs)/costs[-1])

        if len(qs) < 3:
            return PiecewisePolynomial.FirstOrderHold(times, 
                    qs[:, 0:self.nq].T)
        
        return PiecewisePolynomial.CubicShapePreserving(times,
                qs[:, 0:self.nq].T)
        

    def joint_limits(self):
        joint_inds = self.plant.GetJointIndices(self.panda)[:self.nq]
        joint_limits = []
        for i in joint_inds:
            joint = self.plant.get_joint(i)
            joint_limits.append((joint.position_lower_limits()[0],
                                 joint.position_upper_limits()[0]))
        return joint_limits

    def isStateValid(self, state):
        """ wrapper around is_colliding that is passed into ompl"""
        # parse state into q
        q = self.parse_state(state)
        # the state is valid if there are no collisions
        return not self.is_colliding(q)

    def is_colliding(self, q):
        """ returns True if the configuration q results in a collision,
        False otherwise
        """
        # forwards kinematics (setting the position of the panda)
        self.plant.SetPositions(self.plant_context, self.panda, q)
        query_object = self.query_output_port.Eval(self.scene_graph_context)

        return query_object.HasCollisions()


    def parse_state(self, state):
        """ parses ompl RealVectorStateSpace::StateType into a numpy array"""
        q = []
        for i in range(self.nq):
            q.append(state[i])
        return np.array(q)

    def q_to_state(self, space, q):
        """ turns a python list q into an ompl state"""
        state = ob.State(space)
        for i in range(len(q)):
            state[i] = q[i]
        return state
            
        
    def pregrasp_q(self):
        ''' find a good pregrasp pose (self.q_P) given the grasping pose '''
        assert self.q_G is not None
        # find the current grasping pose
        X_Grasp = self.q_to_X_WG(self.q_G)
        
        # retreat in the negative z direction of the gripper frame
        X_Pregrasp = X_Grasp.multiply(RigidTransform(RotationMatrix(), [0, 0, -0.07]))
        q = self.X_WG_to_q(X_Pregrasp)

        return q, X_Pregrasp, X_Grasp
        
    def q_to_X_WG(self, q):
        assert len(q) == self.nq
        
        self.plant.SetPositions(self.plant_context, self.panda, q)
        #self.plant.SetPositions(self.plant_context, self.plant.GetModelInstanceByName("hand"), [-0.04, 0.04])
        
        body = self.plant.GetBodyByName("panda_hand")
        return self.plant.EvalBodyPoseInWorld(self.plant_context, body)
        
        
    def X_WG_to_q(self, X_WG, warm_start = np.array([ 0., 0.55, 0., -1.45, 0., 1.58, 0.])):
        ik = InverseKinematics(self.plant, self.plant_context)
        ik.AddPositionConstraint(
                self.plant.GetFrameByName("panda_hand"),
                np.zeros(3),
                self.plant.world_frame(),
                X_WG.translation() - self.p_tol,
                X_WG.translation() + self.p_tol)
        ik.AddOrientationConstraint(
                self.plant.GetFrameByName("panda_hand"),
                RotationMatrix(),
                self.plant.world_frame(),
                X_WG.rotation(),
                self.theta_tol)
        ik.AddMinimumDistanceConstraint(0.001, 0.1)
        q = ik.q()
        prog = ik.prog()
        #q_nom = np.concatenate((self.q_nominal, np.zeros(2)))
        prog.AddQuadraticErrorCost(np.identity(len(q)), self.q_nominal, q)
        #q_warm = np.concatenate((warm_start, np.zeros(2)))
        prog.SetInitialGuess(q, warm_start)
        result = Solve(prog)
        if not result.is_success():
            print("failed pregrasp ik, returning best result")
        return result.GetSolution(q)
        
        
def make_internal_station():
    builder = DiagramBuilder()
    station = builder.AddSystem(PandaStation(0.001))
    station.SetupTableStation(welded_hand = True)
    station.Finalize()
    
    return station


def random_position():
    return [0.6 + np.random.uniform(-0.05, 0.05), np.random.uniform(-0.1,0.1), 0.15]

def random_orientation():
    rot = RotationMatrix.MakeZRotation(np.random.uniform(-np.pi, np.pi))
    rot = rot.multiply(RotationMatrix.MakeXRotation(-np.pi/2))
    return rot

In [21]:
builder = pydrake.systems.framework.DiagramBuilder()

station = builder.AddSystem(PandaStation(time_step = 0.001))
station.SetupTableStation()
item = "mustard"
station.AddModelFromFile(ycb_resource(item), 
                        RigidTransform(random_orientation(), random_position()))

plant = station.get_multibody_plant()
panda = station.GetPanda()
station.Finalize()

station_context = station.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(station_context)
scene_graph = station.get_scene_graph()
scene_graph_context = station.GetSubsystemContext(scene_graph, station_context)

start_pose = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("panda_hand"))


meshcat = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(builder,
          scene_graph,
          output_port=station.GetOutputPort("query_object"),
          delete_prefix_on_load=True,                                      
          zmq_url=zmq_url)#, role = Role.kProximity)# <- this commented part allows visualization of the collisions


processor = builder.AddSystem(ProcessPointcloud(station, meshcat))
builder.Connect(station.GetOutputPort("camera_left_point_cloud"), processor.GetInputPort("camera_left_pcd"))
builder.Connect(station.GetOutputPort("camera_middle_point_cloud"), processor.GetInputPort("camera_middle_pcd"))
builder.Connect(station.GetOutputPort("camera_right_point_cloud"), processor.GetInputPort("camera_right_pcd"))

internal_station = make_internal_station()

grasp_finder = builder.AddSystem(FindGrasp(internal_station))
picker = builder.AddSystem(Picker(internal_station))

builder.Connect(processor.GetOutputPort("merged_pcd"), grasp_finder.GetInputPort("merged_pcd"))
builder.Connect(processor.GetOutputPort("merged_pcd"), picker.GetInputPort("merged_pcd"))
builder.Connect(grasp_finder.GetOutputPort("panda_grasp_position"), picker.GetInputPort("grasp_position"))
builder.Connect(station.GetOutputPort("panda_position_measured"), picker.GetInputPort("panda_position"))
builder.Connect(picker.GetOutputPort("panda_position_command"), station.GetInputPort("panda_position"))
builder.Connect(picker.GetOutputPort("hand_position_command"), station.GetInputPort("hand_position"))

diagram = builder.Build()

#SVG(pydot.graph_from_dot_data(diagram.GetGraphvizString())[0].create_svg())
simulator = pydrake.systems.analysis.Simulator(diagram)
station_context = station.GetMyContextFromRoot(simulator.get_mutable_context())
#station.GetInputPort("hand_position").FixValue(station_context, [0.08]) # taking the desired hand separation
#station.GetInputPort("panda_position").FixValue(station_context, [0.0, 0.1, 0.0, -1.2, 0, 1.6, 0])


meshcat.start_recording()

simulator.AdvanceTo(10.5)

meshcat.stop_recording()
meshcat.publish_recording()

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6001...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/
Connected to meshcat-server.
INITIALIZATION
Success? False
Infeasible Constraints:
drake::multibody::OrientationConstraint[0]: 2.999900 <= 2.957202 <= 3.000000
TO PREPICK
TO PICK
PICKING
TO POSTPICK


In [4]:
file = open(item + ".html", "w")
file.write(meshcat.vis.static_html())
file.close()

subprocess.check_output(["tar", "vczf", "media/videos/ycb_pick/" + item + ".gz", item + ".html"])
subprocess.check_output(["trash", item +".html"])

b''